In [1]:
#Import Libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot 
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split 
from sklearn import linear_model, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
StandardScaler, MinMaxScaler
import warnings 
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [2]:
#Load the data
df = pd.read_csv("datanew.csv")
df = df.drop(columns=['Unnamed: 0'])

In [3]:
#Subset Data Frame for Enrolled for end of project to predict
df_prediction = df[df['Target'] == "Enrolled"]

In [4]:
#Subset Data Frame for Target (either Dropped Out or Graduated) for future train_test_split
df = df[df['Target'] != "Enrolled"]
df["Target"].value_counts()

Target
Graduate    2209
Dropout     1421
Name: count, dtype: int64

In [5]:
#Family Occupation is telling us a lot of the same things as Family Qualification. Remove Occupation
df = df.drop(columns=["Mother's occupation", "Father's occupation"])

In [6]:
#0 & 1 values for already binary columns
df["Displaced"] = df["Displaced"].map({'yes': 1, 'no': 0})
df["Educational special needs"] = df["Educational special needs"].map({'yes': 1, 'no': 0})
df["Debtor"] = df["Debtor"].map({'yes': 1, 'no': 0})
df["Tuition fees up to date"] = df["Tuition fees up to date"].map({'yes': 1, 'no': 0})
df["Gender"] = df["Gender"].map({'female': 1, 'male': 0})
df["Scholarship holder"] = df["Scholarship holder"].map({'yes': 1, 'no': 0})
df["International"] = df["International"].map({'yes': 1, 'no': 0})

In [7]:
#Create Dummy Features
#Identify Object Columns to Binarize
object_columns = df.select_dtypes(object)
obj = object_columns.columns
#Numerical Columns to merge with the Binarized Columns
num_columns = df.select_dtypes([float, int])
num = num_columns.columns

In [8]:
#Binarize Variables
bdf = pd.get_dummies(df[obj], dtype='int')
#This makes two target variables, we just need to look at Target_Graduate to see 1 for Graduate, 0 for Dropout 
bdf = bdf.drop('Target_Dropout', axis=1)
ndf = df[num]

In [9]:
#Re-Merge Binary Portion of DataFrame and Numerical Portion of DataFrame
df2 = pd.concat([bdf, ndf], axis=1)

In [10]:
#Train, Test, Split
X = df2.drop(columns=["Target_Graduate"], axis = 1)
y = df2[['Target_Graduate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=44)

In [11]:
#Apply StandardScaler to Numerical Data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)